Q1. You are working on a machine learning project where you have a dataset containing numerical and
categorical features. You have identified that some of the features are highly correlated and there are
missing values in some of the columns. You want to build a pipeline that automates the feature
engineering process and handles the missing values.

Design a pipeline that includes the following steps:

Use an automated feature selection method to identify the important features in the dataset

Create a numerical pipeline that includes the following steps

Impute the missing values in the numerical columns using the mean of the column values

Scale the numerical columns using standardisation

Create a categorical pipeline that includes the following steps

Impute the missing values in the categorical columns using the most frequent value of the column

One-hot encode the categorical columns

Combine the numerical and categorical pipelines using a ColumnTransformer

Use a Random Forest Classifier to build the final model

Evaluate the accuracy of the model on the test dataset

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings("ignore")

In [2]:
sns.get_dataset_names()

[]

In [3]:
df=pd.read_csv("diamonds.csv")
df.head()

,carat,cut,color,clarity,depth,table,price,x,y,z
0,0.23,Ideal,E,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,E,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,E,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,I,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,J,SI2,63.3,58.0,335,4.34,4.35,2.75


In [4]:
df["color"].unique()

array(['E', 'I', 'J', 'H', 'F', 'G', 'D'], dtype=object)

In [5]:
from sklearn.preprocessing import LabelEncoder

In [6]:
enc=LabelEncoder()
df["color"]=enc.fit_transform(df["color"])

In [7]:
df["color"].unique()

array([1, 5, 6, 4, 2, 3, 0])

In [8]:
X=df.drop(labels=["color"],axis=1)
X.head()

,carat,cut,clarity,depth,table,price,x,y,z
0,0.23,Ideal,SI2,61.5,55.0,326,3.95,3.98,2.43
1,0.21,Premium,SI1,59.8,61.0,326,3.89,3.84,2.31
2,0.23,Good,VS1,56.9,65.0,327,4.05,4.07,2.31
3,0.29,Premium,VS2,62.4,58.0,334,4.20,4.23,2.63
4,0.31,Good,SI2,63.3,58.0,335,4.34,4.35,2.75


In [9]:
y=df["color"]
y.head()

0    1
1    1
2    1
3    5
4    6
Name: color, dtype: int64

In [10]:
from sklearn.model_selection import train_test_split

In [11]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.3,random_state=42)

In [12]:
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OneHotEncoder
from sklearn.compose import ColumnTransformer

In [13]:
df.shape

(53940, 10)

In [14]:
df.columns

Index(['carat', 'cut', 'color', 'clarity', 'depth', 'table', 'price', 'x', 'y',
       'z'],
      dtype='object')

In [15]:
catcols=['cut', 'clarity']
numcols=['carat', 'depth', 'table', 'price', 'x', 'y','z']

In [16]:
numpipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="mean")),
        ("scaler",StandardScaler())
    ]
)


In [17]:
catpipeline=Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy="most_frequent")),
        ("encoder",OneHotEncoder())
    ]
)

In [18]:
preprocessor=ColumnTransformer([
    ("numpipeline",numpipeline,numcols),
    ("catpipeline",catpipeline,catcols)
])
preprocessor

ColumnTransformer(transformers=[('numpipeline',
                                 Pipeline(steps=[('imputer', SimpleImputer()),
                                                 ('scaler', StandardScaler())]),
                                 ['carat', 'depth', 'table', 'price', 'x', 'y',
                                  'z']),
                                ('catpipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('encoder', OneHotEncoder())]),
                                 ['cut', 'clarity'])])

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 53940 entries, 0 to 53939
Data columns (total 10 columns):
 #   Column   Non-Null Count  Dtype  
---  ------   --------------  -----  
 0   carat    53940 non-null  float64
 1   cut      53940 non-null  object 
 2   color    53940 non-null  int64  
 3   clarity  53940 non-null  object 
 4   depth    53940 non-null  float64
 5   table    53940 non-null  float64
 6   price    53940 non-null  int64  
 7   x        53940 non-null  float64
 8   y        53940 non-null  float64
 9   z        53940 non-null  float64
dtypes: float64(6), int64(2), object(2)
memory usage: 4.1+ MB


In [20]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [21]:
from sklearn.ensemble import RandomForestClassifier

In [22]:
models={
    "Random Forest":RandomForestClassifier()
}

In [23]:
from sklearn.metrics import accuracy_score

In [24]:
def evaluate_model(X_train,y_train,X_test,y_test,models):

    report = {}
    for i in range(len(models)):
        model = list(models.values())[i]
        # Train model
        model.fit(X_train,y_train)



        # Predict Testing data
        y_test_pred =model.predict(X_test)

        # Get accuracy for test data prediction

        test_model_score = accuracy_score(y_test,y_test_pred)

        report[list(models.keys())[i]] =  test_model_score



    return report

In [25]:
evaluate_model(X_train,y_train,X_test,y_test,models)

{'Random Forest': 0.5162526263749846}

Q2. Build a pipeline that includes a random forest classifier and a logistic regression classifier, and then
use a voting classifier to combine their predictions. Train the pipeline on the iris dataset and evaluate its
accuracy.

In [27]:
df=pd.read_csv("iris.csv")
df.head()

,Species,Sepallen,Sepalwid,Petallen,Petalwid
0,1,5.1,3.5,1.4,0.2
1,1,4.9,3.0,1.4,0.2
2,1,4.7,3.2,1.3,0.2
3,1,4.6,3.1,1.5,0.2
4,1,5.0,3.6,1.4,0.2


In [41]:
X=df.drop(labels=["Species"],axis=1)
X.head()

,Sepallen,Sepalwid,Petallen,Petalwid
0,5.1,3.5,1.4,0.2
1,4.9,3.0,1.4,0.2
2,4.7,3.2,1.3,0.2
3,4.6,3.1,1.5,0.2
4,5.0,3.6,1.4,0.2


In [42]:
y=df["Species"]
y.head()

0    1
1    1
2    1
3    1
4    1
Name: Species, dtype: int64

In [43]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,  
                                                    y,  
                                                    test_size = 0.20,  
                                                    random_state = 42)

In [30]:
from sklearn.ensemble import VotingClassifier

In [31]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression

In [45]:
models=[]
models.append(("RandomForest",RandomForestClassifier()))
models.append(("LogisticRegression",LogisticRegression()))

In [46]:
votehard=VotingClassifier(models,voting ='hard')

In [47]:
votehard.fit(X_train, y_train) 
y_pred = votehard.predict(X_test) 

In [48]:
score = accuracy_score(y_test, y_pred) 
print("Hard Voting Score % d" % score) 

Hard Voting Score  1


In [49]:
votehard=VotingClassifier(models,voting ='soft')

In [50]:
votehard.fit(X_train, y_train) 
y_pred = votehard.predict(X_test) 

In [51]:
score = accuracy_score(y_test, y_pred) 
print("Hard Voting Score % d" % score) 

Hard Voting Score  1
